# Example Usage

In [2]:
import sys
from pathlib import Path

home = str(Path.home())

In [3]:
import pandas as pd
import numpy as np

In [4]:
import pyrankability

Looking for packages in home directory


## SCIP Verification

Please see the README.md for more details

In [5]:
import os
os.environ['PATH']

'/opt/tljh/user/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin'

In [6]:
os.environ['PATH']=os.environ['PATH']+":/usr/local/SCIPOptSuite/bin"
os.environ['PATH']=os.environ['PATH']+":"+os.path.expanduser('~/ranking_toolbox/scip')

In [7]:
%%bash
which scip_collect.sh
which scip_count.sh
which scip

/home/jupyter-pander14/ranking_toolbox/scip/scip_collect.sh
/home/jupyter-pander14/ranking_toolbox/scip/scip_count.sh
/usr/local/SCIPOptSuite/bin/scip


## Create dataset

In [8]:
n=8
D=np.zeros((n,n))
D[np.triu_indices(n,1)]=1
D[[5,3,7]] = 1-D[[5,3,7]]
D=pd.DataFrame(D,dtype=int)
D

,0,1,2,3,4,5,6,7
0,0,1,1,1,1,1,1,1
1,0,0,1,1,1,1,1,1
2,0,0,0,1,1,1,1,1
3,1,1,1,1,0,0,0,0
4,0,0,0,0,0,1,1,1
5,1,1,1,1,1,1,0,0
6,0,0,0,0,0,0,0,1
7,1,1,1,1,1,1,1,1


### Solve the problem first with the include_model flag

In [21]:
obj_lop,details_lop = pyrankability.rank.solve(D,method='lop',include_model=True)

here


In [13]:
print('Objective values:',obj_lop,details_lop['obj'])

Objective values: 12 25.0


### Collect solutions

In [54]:
model = details_lop['model']
model_file = pyrankability.common.write_model(model)
results = pyrankability.search.scip_collect(D,model_file,max_num_solutions=10)

Running: scip_collect.sh /tmp/tmp3jla3hai/model.mps.fixed.mps /tmp/tmpxveoi6nk/model.solutions 10
STDOUT:
Model file: /tmp/tmp3jla3hai/model.mps.fixed.mps
Solutions file: /tmp/tmpxveoi6nk/model.solutions
SCIP version 7.0.0 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: SoPlex 5.0.0] [GitHash: 0bc4dc9c65]
Copyright (C) 2002-2020 Konrad-Zuse-Zentrum fuer Informationstechnik Berlin (ZIB)

External codes: 
  SoPlex 5.0.0         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 6535a3c8]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (www.coin-or.org/CppAD)
  ZLIB 1.2.11          General purpose compression library by J. Gailly and M. Adler (zlib.net)
  GMP 6.2.0            GNU Multiple Precision Arithmetic Library developed by T. Granlund (gmplib.org)
  ZIMPL 3.3.9          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  PaPILO 1.0.0         parallel pr

/home/jupyter-pander14/ranking_toolbox/pyrankability/search.py:740: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  n = int((-b + np.sqrt(b**2 - 4*a*c))/(2*a) + 1)


In [56]:
results['perms']

[(7, 5, 3, 0, 1, 2, 4, 6),
 (7, 5, 0, 3, 1, 2, 4, 6),
 (7, 0, 5, 3, 1, 2, 4, 6),
 (7, 5, 0, 1, 3, 2, 4, 6),
 (7, 0, 5, 1, 3, 2, 4, 6),
 (0, 7, 5, 3, 1, 2, 4, 6),
 (0, 7, 5, 1, 3, 2, 4, 6),
 (0, 1, 2, 4, 6, 7, 5, 3),
 (7, 0, 1, 2, 5, 4, 6, 3),
 (7, 0, 1, 2, 5, 4, 3, 6)]

In [59]:
pd.DataFrame(results['rankings'],dtype=int)

,0,1,2,3,4,5,6,7
0,3,4,5,2,6,1,7,0
1,2,4,5,3,6,1,7,0
2,1,4,5,3,6,2,7,0
3,2,3,5,4,6,1,7,0
4,1,3,5,4,6,2,7,0
5,0,4,5,3,6,2,7,1
6,0,3,5,4,6,2,7,1
7,0,1,2,7,3,6,4,5
8,1,2,3,7,5,4,6,0
9,1,2,3,6,5,4,7,0


In [47]:
print('Number of solutions found:',len(results['xs']))

Number of solutions found: 10


In [48]:
print('X*')
results['xstar']

X*


,0,1,2,3,4,5,6,7
0,0.0,1.0,1.0,0.9,1.0,0.7,1.0,0.3
1,0.0,0.0,1.0,0.6,1.0,0.3,1.0,0.1
2,0.0,0.0,0.0,0.3,1.0,0.3,1.0,0.1
3,0.1,0.4,0.7,0.0,0.7,0.0,0.8,0.0
4,0.0,0.0,0.0,0.3,0.0,0.1,1.0,0.1
5,0.3,0.7,0.7,1.0,0.9,0.0,0.9,0.0
6,0.0,0.0,0.0,0.2,0.0,0.1,0.0,0.1
7,0.7,0.9,0.9,1.0,0.9,1.0,0.9,0.0


In [49]:
print('Objective values:')
pd.Series(results['objs'])

Objective values:


0    25.0
1    25.0
2    25.0
3    25.0
4    25.0
5    25.0
6    25.0
7    25.0
8    25.0
9    25.0
dtype: float64